In [142]:
import pandas as pd
from datetime import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [143]:
dates = pd.date_range(start='2022-02-08', end = '2022-04-10', freq='D').to_frame().reset_index(drop=True)
dates.columns = ['date']
dates['date'] = dates['date'].apply(lambda x: x.strftime('%Y%m%d'))
dates = dates.date.to_list()

In [144]:
schedule = pd.DataFrame()
for date in dates:
    try:
        df = pd.read_html(f'https://www.cbssports.com/nba/schedule/{date}/')[0]
        df = df[['Away','Home','Time / TV']]
        df['date'] = date
        schedule = pd.concat([schedule, df])
    except ValueError:
        continue
schedule.date = pd.to_datetime(schedule.date)

In [145]:
schedule = schedule.sort_values(by='date')

In [146]:
def name_fix(row):
    all_teams = pd.read_csv('data/all_teams.csv')
    all_teams = all_teams['full_name'].to_list()
    if row == 'L.A. Lakers':
        return 'Los Angeles Lakers'
    if row == 'L.A. Clippers':
        return 'Los Angeles Clippers'
    if row == 'Golden St.':
        return 'Golden State Warriors'
    for val in all_teams:
        if val.split()[0] == row:
            return val
    return row

In [147]:
schedule.Away = schedule.Away.apply(name_fix)
schedule.Home = schedule.Home.apply(name_fix)

In [148]:
schedule.to_csv('data/schedule.csv', index=False)